In [1]:
import fitbit
import gather_keys_oauth2 as Oauth2
import pandas as pd
import datetime
from datetime import timedelta
import numpy as np
import warnings
warnings.filterwarnings('ignore')



CLIENT_ID='<Your CLIENT_ID >'
CLIENT_SECRET='<Your CLIENT_SECRET>'


server=Oauth2.OAuth2Server(CLIENT_ID, CLIENT_SECRET)
server.browser_authorize()
ACCESS_TOKEN=str(server.fitbit.client.session.token['access_token'])
REFRESH_TOKEN=str(server.fitbit.client.session.token['refresh_token'])
auth2_client=fitbit.Fitbit(CLIENT_ID,CLIENT_SECRET,oauth2=True,access_token=ACCESS_TOKEN,refresh_token=REFRESH_TOKEN)

#NEed this seeparately because version 1 doesn't give sleep breakdown
auth2_client_new = fitbit.Fitbit(CLIENT_ID, CLIENT_SECRET, oauth2=True, access_token=ACCESS_TOKEN, refresh_token=REFRESH_TOKEN, system='en_GB')
auth2_client_new.API_VERSION = 1.2

[04/Apr/2020:23:38:16] ENGINE Listening for SIGTERM.
[04/Apr/2020:23:38:16] ENGINE Bus STARTING
[04/Apr/2020:23:38:16] ENGINE Set handler for console events.
[04/Apr/2020:23:38:16] ENGINE Started monitor thread 'Autoreloader'.
[04/Apr/2020:23:38:16] ENGINE Serving on http://127.0.0.1:8080
[04/Apr/2020:23:38:16] ENGINE Bus STARTED


127.0.0.1 - - [04/Apr/2020:23:38:18] "GET /?code=14335af193f505e4684f4557b663ad3026b6e053&state=klXlTkk69bUR34ecf35ej76hjzFNe9 HTTP/1.1" 200 122 "" "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.162 Safari/537.36 Edg/80.0.361.109"


[04/Apr/2020:23:38:19] ENGINE Bus STOPPING
[04/Apr/2020:23:38:19] ENGINE HTTP Server cherrypy._cpwsgi_server.CPWSGIServer(('127.0.0.1', 8080)) shut down
[04/Apr/2020:23:38:19] ENGINE Stopped thread 'Autoreloader'.
[04/Apr/2020:23:38:19] ENGINE Removed handler for console events.
[04/Apr/2020:23:38:19] ENGINE Bus STOPPED
[04/Apr/2020:23:38:19] ENGINE Bus EXITING
[04/Apr/2020:23:38:19] ENGINE Waiting for child threads to terminate...
[04/Apr/2020:23:38:19] ENGINE Bus EXITED
[04/Apr/2020:23:38:19] ENGINE Waiting for thread Thread-19.


In [29]:
DATE_FORMAT = '%Y-%m-%d'
    
def prettyPrintDate(date):
    return date.strftime(DATE_FORMAT);


def safeGet(obj, key, defaultVal = np.nan):
    return obj.get(key, defaultVal)  


def filterArrObj(arrList, keyName, keyValue):
    for arrItem in arrList:
        if arrItem.get(keyName) == keyValue:
            return arrItem
    return {}

def getActivities(date):
    activitiyResponse = auth2_client.activities(date=date)
    
    activitySummary = activitiyResponse['summary'];
    activityData = {
        'Calories Burned':safeGet(activitySummary,'caloriesOut'),
        'Calories BMR': safeGet(activitySummary,'caloriesBMR'),
        'Resting Heart Rate':safeGet(activitySummary,'restingHeartRate'),
        'Steps':safeGet(activitySummary,'steps'),
        'Distance (Km)':filterArrObj(activitySummary.get('distances', []), 'activity', 'total').get('distance', np.nan),
        'Elevation (Ft)':activitySummary['elevation'],
        'Floors':activitySummary['floors'],
        'Minutes Sedentary':activitySummary['sedentaryMinutes'],
        'Minutes Lightly Active':activitySummary['lightlyActiveMinutes'],
        'Minutes Fairly Active':activitySummary['fairlyActiveMinutes'],
        'Minutes Very Active':activitySummary['veryActiveMinutes'],
        'Activity Calories': activitySummary['activityCalories'],
        'Active Score': activitySummary['activeScore'],
        'Cardio minutes': filterArrObj(activitySummary.get('heartRateZones', []), 'name', 'Cardio').get('minutes', np.nan),
        'Cardio calories': filterArrObj(activitySummary.get('heartRateZones', []), 'name', 'Cardio').get('caloriesOut', np.nan),
        'Fat Burn minutes': filterArrObj(activitySummary.get('heartRateZones', []), 'name', 'Fat Burn').get('minutes', np.nan),
        'Fat Burn calories': filterArrObj(activitySummary.get('heartRateZones', []), 'name', 'Fat Burn').get('caloriesOut', np.nan),
        'Peak minutes': filterArrObj(activitySummary.get('heartRateZones', []), 'name', 'Peak').get('minutes', np.nan),
        'Peak calories': filterArrObj(activitySummary.get('heartRateZones', []), 'name', 'Peak').get('caloriesOut', np.nan),
        'Normal Cardio minutes': filterArrObj(activitySummary.get('heartRateZones', []), 'name', 'Out of Range').get('minutes', np.nan),
        'Normal Cardio calories': filterArrObj(activitySummary.get('heartRateZones', []), 'name', 'Out of Range').get('caloriesOut', np.nan),
    }
    
    return activityData

def getSleep(date):
    sleepResponse = auth2_client_new.sleep(date=date)
    
    sleepData = {}
    for sleepLog in sleepResponse.get('sleep', []):
        if sleepLog.get('isMainSleep'):
            sleepLevelsSummary = sleepLog.get('levels', {}).get('summary', {})
            
            sleepData['Sleep Efficiency'] = safeGet(sleepLog, 'efficiency')
            sleepData['Minutes Asleep'] = safeGet(sleepLog, 'minutesAsleep')
            sleepData['Minutes to fall asleep'] = safeGet(sleepLog, 'minutesToFallAsleep')
            sleepData['Sleep Start time'] = safeGet(sleepLog, 'startTime')
            sleepData['Sleep End time'] = safeGet(sleepLog, 'endTime')
            
            sleepData['Time in bed'] = safeGet(sleepLog, 'timeInBed')
            
            sleepData['Minutes Deep sleep'] = safeGet(sleepLevelsSummary.get('deep', {}), 'minutes')
            sleepData['Deep sleep count'] = safeGet(sleepLevelsSummary.get('deep', {}), 'count')
#             sleepData['% Deep sleep'] = percent(safeGet(sleepData, 'Minutes Deep sleep', 0)/safeGet(sleepData, 'Time in bed', 0))
            
            sleepData['Minutes Light sleep'] = safeGet(sleepLevelsSummary.get('light', {}), 'minutes')
            sleepData['Light sleep count'] = safeGet(sleepLevelsSummary.get('light', {}), 'count')
#             sleepData['% Light sleep'] = percent(safeGet(sleepData, 'Minutes Light sleep', 0)/safeGet(sleepData, 'Time in bed', 0))
            
            sleepData['Minutes REM sleep'] = safeGet(sleepLevelsSummary.get('rem', {}), 'minutes')
            sleepData['REM sleep count'] = safeGet(sleepLevelsSummary.get('rem', {}), 'count')
#             sleepData['% REM sleep'] = percent(safeGet(sleepData, 'Minutes REM sleep', 0)/safeGet(sleepData, 'Time in bed', 0))
            
            sleepData['Minutes Asleep'] = sleepData['Minutes Deep sleep'] + sleepData['Minutes Light sleep'] + sleepData['Minutes REM sleep']
            sleepData['Minutes Awake'] = safeGet(sleepLevelsSummary.get('wake', {}), 'minutes')
            sleepData['Minutes Awake count'] = safeGet(sleepLevelsSummary.get('wake', {}), 'count')            
    return sleepData

def getDateData(date):
    weekDayNum = date.isoweekday()
    return {
        'Day of Week': weekDayNum,
        'Is Weekday': weekDayNum<6,
        'Is Weekend': weekDayNum>5,
        'Date': prettyPrintDate(date)
    };

def fetchAllData(date):
    dateStr = prettyPrintDate(date)
    
    print("Fetching fitbit data for: " + dateStr)
    
    nextDate = date + timedelta(days=1)
    sleepData = getSleep(prettyPrintDate(nextDate))
     
    activitiesData = getActivities(dateStr)
    dateData = getDateData(date)
    
    
    mergedData = {**sleepData, **activitiesData, **dateData} #merging the dictionaries together
    return mergedData


In [44]:
# initiating an empty dataframe to store the final results
database = pd.DataFrame()


def fetchAndAppendToDb(date, database):    
    mergedData = fetchAllData(date)
    database = database.append(mergedData, ignore_index=True)
    return database;

def fetchData(database):    
    #here we are extracting the data for the last 1 week
    StartDate = (datetime.datetime.today() - datetime.timedelta(9)) #date from which you want to start extracting data
    EndDate = (datetime.datetime.today() - datetime.timedelta(1)) #date upto which you want to extract data 
    dateToFetch = StartDate + timedelta(days=1)
    while dateToFetch < EndDate:
        database = fetchAndAppendToDb(dateToFetch, database)
        dateToFetch = dateToFetch + timedelta(days=1)
    print("----------------------------------------------")
    print("Data fill completed! 👍👍")
    return database

In [45]:
database = fetchData(database)
database

# database.to_excel('Fitbit_data.xlsx')

Fetching fitbit data for: 2020-03-28
Fetching fitbit data for: 2020-03-29
Fetching fitbit data for: 2020-03-30
Fetching fitbit data for: 2020-03-31
Fetching fitbit data for: 2020-04-01
Fetching fitbit data for: 2020-04-02
Fetching fitbit data for: 2020-04-03
----------------------------------------------
Data fill completed! 👍👍


,Active Score,Activity Calories,Calories BMR,Calories Burned,Cardio calories,Cardio minutes,Date,Day of Week,Deep sleep count,Distance (Km),...,Normal Cardio minutes,Peak calories,Peak minutes,REM sleep count,Resting Heart Rate,Sleep Efficiency,Sleep End time,Sleep Start time,Steps,Time in bed
0,-1.0,1236.0,1746.0,2791.0,0.00000,0.0,2020-03-28,6.0,4.0,3.53,...,1350.0,0.0,0.0,7.0,53.0,91.0,2020-03-29T09:42:00.000,2020-03-29T01:32:00.000,7817.0,490.0
1,-1.0,1261.0,1746.0,2826.0,11.03648,1.0,2020-03-29,7.0,5.0,3.55,...,1334.0,0.0,0.0,13.0,54.0,94.0,2020-03-30T09:36:00.000,2020-03-30T00:49:00.000,7864.0,527.0
2,-1.0,1304.0,1746.0,2783.0,0.00000,0.0,2020-03-30,1.0,6.0,3.57,...,1369.0,0.0,0.0,12.0,55.0,92.0,2020-03-31T09:50:30.000,2020-03-31T00:59:30.000,7899.0,531.0
3,-1.0,730.0,1746.0,2338.0,0.00000,0.0,2020-03-31,2.0,4.0,2.25,...,1372.0,0.0,0.0,8.0,57.0,84.0,2020-04-01T09:40:30.000,2020-04-01T01:13:30.000,4985.0,507.0
4,-1.0,1026.0,1746.0,2551.0,0.00000,0.0,2020-04-01,3.0,6.0,3.14,...,1371.0,0.0,0.0,10.0,55.0,90.0,2020-04-02T09:56:30.000,2020-04-02T01:08:00.000,6954.0,528.0
5,-1.0,931.0,1746.0,2543.0,0.00000,0.0,2020-04-02,4.0,5.0,2.84,...,1423.0,0.0,0.0,8.0,55.0,89.0,2020-04-03T09:40:30.000,2020-04-03T01:02:30.000,6277.0,518.0
6,-1.0,840.0,1746.0,2449.0,0.00000,0.0,2020-04-03,5.0,5.0,2.31,...,1381.0,0.0,0.0,6.0,55.0,90.0,2020-04-04T09:23:30.000,2020-04-04T02:15:00.000,5122.0,428.0


The next phase of this project involves performing Exploratory Data Analysis (EDA) of the collected data. Refer to the link provided below to check oout the EDA: 

**[Fitbit Exploratory Data Analysis](https://github.com/Arnab-Rajkhowa/Fitbit-Data-Analysis/blob/master/Fitbit%20Exploratory%20Data%20Analysis.ipynb)**